<a href="https://colab.research.google.com/github/aidot-kr/AISecurity/blob/master/6_Similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Similarity

1. 코사인 유사도(Cosine Similarity)
 - 두 벡터 사이의 각도의 코사인을 측정 하여 유사성을 계산
 - 코사인 유사성을 사용하여 문장을 벡터로 변환
 - 1) BoW(TF) : 일반적인 문서 유사도 비교에 좋음
 - 2) TF-IDF : 검색의 목적에는 유용함
 - 3) Word2Vec : 문맥기반의 유사성 비교에 용이

In [2]:
# 코사인 유사도 비교대상 문서
doc_A = "카카오페이가 개인정보가 털려 ㅋㅋ 본인도 모르는 카카오 결제가 되는 등 피해를 본 이용자에게 수사 결과가 나오기 전에 먼저 보상하기로 하는 등 강화된 이용자 보호 정책을 도입한다."
doc_B = "카카오페이가 내달 도입 예정인 새 정책의 핵심은 개인정보 도용 등 부정 결제로 인한 피해 사례를 접수하면 외부 기관의 수사 의뢰와는 별개로 자체 사고 조사를 해서 선량한 피해자로 판명되면 먼저 보상하겠다는 것이다."
doc_C = "지금까지 고객이 카카오 휴대폰 파손에 따른 보험 혜택을 받으려면 AS센터를 방문해 수리를 받고, 종이로 된 수리 명세서와 영수증을 다시 보험사 측에 제출해야 했다."

documents = [doc_A, doc_B, doc_C]

In [3]:
# 한글 불용어 처리를 위한 함수
import re
def clean_sentense(txt):
    pattern = '(\d\d\d-\d\d\d\d-\d\d\d\d)' # 전화번호 제거 (000-0000-0000),\d: 숫자 1개
    txt = re.sub(pattern=pattern, repl='', string=txt)          
    pattern = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)' # E-mail제거, a-z 사이의 문자, 
    txt = re.sub(pattern=pattern, repl='', string=txt)
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+' # URL제거
    txt = re.sub(pattern=pattern, repl='', string=txt)
    pattern = '([ㄱ-ㅎㅏ-ㅣ]+)'  # 한글 자음, 모음 제거
    txt = re.sub(pattern=pattern, repl='', string=txt)
    pattern = '<[^>]*>'         # HTML 태그 제거
    txt = re.sub(pattern=pattern, repl='', string=txt)
    pattern = '[^\w\s]'         # 특수기호제거
    txt = re.sub(pattern=pattern, repl='', string=txt)
    
    return txt   

In [6]:
!pip3 install konlpy

     |████████████████████████████████| 19.4MB 1.1MB/s 
     |████████████████████████████████| 3.8MB 62.4MB/s 
     |████████████████████████████████| 92kB 10.9MB/s 
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [7]:
# 형태소 분석기를 사용하여 문서 처리 (특정 POS 추출, 1글자 제외)
from konlpy.tag import Kkma
kkma = Kkma()

# 명사와 동사만 추출하는 함수
def extPOS(x):
    allowed_postags=['NNG', 'NNP', 'VV']
    texts = kkma.pos(x) 
    texts_out = []
    #print(texts)
    
    for sent in texts:         
        if sent[1] in allowed_postags:
            texts_out.append(sent[0])
    
    return " ".join(texts_out)

In [8]:
documents = [clean_sentense(sent) for sent in documents ]
documents = [extPOS(doc) for doc in documents ]

documents

['카카오 이 개인 정보 털리 본인 모르 카카오 결제 되 피해 보 이용자 수사 결과 나오 전 보상 하 등 강화 이용자 보호 정책 도입',
 '카카오 이 내달 도입 예정 새 정책 핵심 개인 정보 도용 부정 결제 인하 피해 사례 접수 외부 기관 수사 의뢰 별개 자체 사고 조사 하 선량 피해자 판명 보상',
 '지금 고객 카카오 휴대폰 파손 따르 보험 혜택 받으 센터 방문 수리 받 종이 되 수리 명세서 영수증 보험사 제출 하']

In [9]:
# Scikit Learn 라이브러리 선언
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

# Document Term Matrix 생성, Bag of Words 를 사용하여 기간 빈도를 계산함
count_vectorizer = CountVectorizer()
sparse_matrix = count_vectorizer.fit_transform(documents)

# (선택) Sparse Matrix를 Pandas Dataframe으로 변환(조회용) 
doc_term_matrix = sparse_matrix.todense()
df = pd.DataFrame(doc_term_matrix, 
                  columns=count_vectorizer.get_feature_names(), 
                  index=['doc_A', 'doc_B', 'doc_C'])
df

,강화,개인,결과,결제,고객,기관,나오,내달,도용,도입,따르,명세서,모르,받으,방문,별개,보상,보험,보험사,보호,본인,부정,사고,사례,선량,센터,수리,수사,영수증,예정,외부,의뢰,이용자,인하,자체,접수,정보,정책,제출,조사,종이,지금,카카오,털리,파손,판명,피해,피해자,핵심,혜택,휴대폰
doc_A,1,1,1,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,2,0,0,0,1,1,0,0,0,0,2,1,0,0,1,0,0,0,0
doc_B,0,1,0,1,0,1,0,1,1,1,0,0,0,0,0,1,1,0,0,0,0,1,1,1,1,0,0,1,0,1,1,1,0,1,1,1,1,1,0,1,0,0,1,0,0,1,1,1,1,0,0
doc_C,0,0,0,0,1,0,0,0,0,0,1,1,0,1,1,0,0,1,1,0,0,0,0,0,0,1,2,0,1,0,0,0,0,0,0,0,0,0,1,0,1,1,1,0,1,0,0,0,0,1,1


In [10]:
# Compute Cosine Similarity
from sklearn.metrics.pairwise import cosine_similarity
print(cosine_similarity(df, df))
#> [[ 1.          0.48927489  0.37139068]
#>  [ 0.48927489  1.          0.38829014]
#>  [ 0.37139068  0.38829014  1.        ]]

[[1.         0.40128618 0.09325048]
 [0.40128618 1.         0.04303315]
 [0.09325048 0.04303315 1.        ]]


2. Jaccard 유사성
 - 비교 대상 2개 문서의 교집합의 크기를 2개 문서의 합집합 크기로 나눈 것으로 정의

In [11]:
# Jaccard 유사도 비교 함수 
def get_jaccard_sim(str1, str2):     
    a = set(str1.split())      
    b = set(str2.split())
    c = a.intersection(b) # a와 b의 교집합을 구함
    return float(len(c)) / (len(a) + len(b) - len(c))

In [12]:
documents[0]

'카카오 이 개인 정보 털리 본인 모르 카카오 결제 되 피해 보 이용자 수사 결과 나오 전 보상 하 등 강화 이용자 보호 정책 도입'

In [13]:
b=set(documents[0].split())  # 중복을 허용하지 않는 집햡자료형으로 변환 
print(b)

{'피해', '결과', '나오', '하', '본인', '정책', '카카오', '모르', '수사', '도입', '보상', '강화', '보', '정보', '이용자', '결제', '이', '등', '털리', '보호', '개인', '되', '전'}


In [14]:
# 문서A와 문서B의 유사도 계산
get_jaccard_sim(documents[0], documents[1])

0.2619047619047619

In [15]:
# 문서B와 문서C의 유사도 계산
get_jaccard_sim(documents[1], documents[2])

0.041666666666666664

In [16]:
# 문서A와 문서C의 유사도 계산
get_jaccard_sim(documents[0], documents[2])

0.075